## Managing secrets

We would need a way to manage secrets, ideally with the following list of features:


1. Ability to source key-value based secrets from multiple sources, including Vault (production), File and Env Vars (in order of preference)
2. Ability to establish trust with Vault using

    a. Username/password entered by humans to prove their identity

    b. Workload-based identity when running in kuberenetes
    
    

In [2]:
from dataclasses import dataclass, fields
import json
import logging
from logging.config import dictConfig
import os

from dataclasses_json import dataclass_json
import hvac

from experiment_utils.constants import LOGGING_CONFIG, BASE_DIR

In [3]:
dictConfig(LOGGING_CONFIG)
logger = logging.getLogger(__name__)


@dataclass_json
@dataclass
class Secrets:
    AZURE_BLOBSTORE_CONTAINER_URL: str

    @classmethod
    def try_load_from_secrets_file(cls) -> 'Secrets':
        default_location = os.path.abspath(os.path.join(BASE_DIR, '.local', 'secrets.json'))
        
        secrets_file_location = os.getenv('GLOBAL_SECRETS_FILE', default_location)

        if not os.path.exists(secrets_file_location):
            raise FileNotFoundError(f"Could not find secrets file at: {secrets_file_location}. Set env variable GLOBAL_SECRETS_FILE to the path to secrets.json.")
        
        try:
            with open(secrets_file_location, 'r') as f:
                return cls.from_json(f.read())
        except FileNotFoundError:
            raise ValueError(f"Could not find secrets file at: {secrets_file_location}.")
    
    @classmethod
    def try_load_from_env_vars(cls) -> 'Secrets':
        env_var_names = { field.name for field in fields(cls) }

        return cls.from_dict({k: v for k, v in os.environ.items() if k in env_var_names})

    
    @classmethod
    def load(cls) -> 'Secrets':
        secret_loaders = [Secrets.try_load_from_secrets_file, Secrets.try_load_from_env_vars]
        
        for loader in secret_loaders:
            try:
                return loader()
            except Exception:
                logger.warn("Failed to load secrets from loader: %s", loader.__name__)
                
        raise RuntimeError("Could not load secrets.")



In [15]:
import getpass
password = getpass.getpass()

 ········


In [16]:
client = hvac.Client(url='https://secrets.shkhr.ovh')

In [17]:
client.auth.userpass.login(username='shikhar', password=password)

{'request_id': '3fece5d8-daae-ad93-048c-0184a278c0e6',
 'lease_id': '',
 'renewable': False,
 'lease_duration': 0,
 'data': None,
 'wrap_info': None,
 'warnings': None,
 'auth': {'client_token': 'hvs.CAESIPX90_L0tiEtH6PngfYxQ4UPJvPZjH57hBBLuvMlxz7AGh4KHGh2cy5rZkpXWFc0dFZTeEZ2VE4yQWtseUVmUHE',
  'accessor': '5z8s3JOSKYpUUeEp95f9WwWn',
  'policies': ['admins', 'default'],
  'token_policies': ['admins', 'default'],
  'metadata': {'username': 'shikhar'},
  'lease_duration': 604800,
  'renewable': True,
  'entity_id': 'c368d9b5-1a45-4cac-c6a7-79eeb9531f0b',
  'token_type': 'service',
  'orphan': True,
  'mfa_requirement': None,
  'num_uses': 0}}

In [18]:
client.is_authenticated()

True

In [ ]:
client.secrets